In [1]:
#' Reverse ID Lookup
#'
#' Looks up a unique identifier for an entity from an authoritative source 
#' by using multiple non-unique and/or partially unreliable individual identifiers
#'
#' Author: Jacob J. Walker
#'
#' Note: Currently this is a linear script, but it will be changed into a function in the future to be released as an R Package
#' Also, it is very likely that this could be done much more efficiently, but this works for now

In [2]:
# Initialize Packages                                                                                   # Code Block Info: 2019-09-17 From Template
# Automatically installs packages that do not exist, and loads them also
# Note: Pacman is no longer used, because it sometimes didn't work right

if (!require('RSelenium')) install.packages('RSelenium'); library('RSelenium')          # Selenium Web Browser Automation Framework
if (!require('keyring')) install.packages('keyring'); library('keyring')                # Used to load passwords from the system instead of being in plain text
if (!require('tidyverse')) install.packages('tidyverse'); library('tidyverse')          # Loads dplyr, and all the other parts of tidyverse
if (!require('lubridate')) install.packages('lubridate'); library('lubridate')          # Explicitly loaded from tidyverse to be used in place of the Base date functions, where appropriate
if (!require('batman')) install.packages('batman'); library('batman')                   # Includes the to_logical function, which is very useful for ETL work on Booleanesq columns
if (!require('stringi')) install.packages('stringi'); library('stringi')                # Includes some string replacement functions that are easier than tidyverse
if (!require('xlsx')) install.packages('xlsx'); library('xlsx')                         # Used to work with Excel files, but there is some type of error happening with Java at the moment
if (!require('XML')) install.packages('XML'); library('XML')                            # Used for web scraping HTML, especially tables
if (!require('urltools')) install.packages('urltools'); library('urltools')             # Used for parsing URLs
# if (!require('RPostgres')) install.packages('RPostgres'); library('RPostgres')          # A DBI Compliant Interface for PostgreSQL
if (!require('odbc')) install.packages('odbc'); library('odbc')                         # A DBI Interface for ODBC (Used for MS SQL)

Loading required package: RSelenium
Loading required package: keyring
Loading required package: tidyverse
-- Attaching packages --------------------------------------- tidyverse 1.3.0 --
v ggplot2 3.2.1     v purrr   0.3.3
v tibble  2.1.3     v dplyr   0.8.3
v tidyr   1.0.0     v stringr 1.4.0
v readr   1.3.1     v forcats 0.4.0
-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()
Loading required package: lubridate

Attaching package: 'lubridate'

The following object is masked from 'package:base':

    date

Loading required package: batman
Loading required package: stringi
Loading required package: xlsx
Loading required package: XML
Loading required package: urltools
Loading required package: odbc


In [3]:
# Variables/Objects
#
# Data Frames / Tibbles
# ---------------------
# source - Source file which needs the unique identifier
# target - Target file where transformed source file will be saved
# lookup_table - table of unique identifiers and other identifiers
#
# Columns (Use with !! and := notation for tibbles)
# -------
# target_uid_column - name of the column that will contain the unique identifier in the target
# lookup_uid_column - name of hte column that contains the unique identifier in the lookup_table
# source_nuid_columns - vector of names of the columns that will be used in combination to do the reverse lookup from the source
# lookup_nuid_columns - vector of names of the columuns that will be used in combination to do the reverse lookup from the lookup_table

In [4]:
# Load source and lookup_table, including lookup column names

source <- tibble(id = "", 
                 first_name =  c("Barack",     "George",     "Bill",       "George",     "Ronald",     "Jimmy"     ),
                 middle_name = c("Hussein",    "Walker",     "",           "H.W.",       "Wilson",     ""          ),
                 last_name  =  c("Obama",      "Bush",       "Clinton",    "Bush",       "Reagon",     "Carter"    ),
                 dob = as.Date(c("1961-08-04", "1946-07-06", "1946-08-19", "1924-06-12", "1911-02-06", "1924-10-01")))

lookup_table <- tibble(num = c(44, 43, 42, 41, 40, 39),
                       fname = c("Barack",     "George",     "William",       "George",               "Ronald",     "James"     ),
                       mname = c("Hussein",    "Walker",     "Jefferson",     "Herbert Walker",       "Wilson",     "Earl"      ),
                       lname = c("Obama",      "Bush",       "Clinton",       "Bush",                 "Reagon",     "Carter"    ),
               bdate = as.Date(c("1961-08-04", "1946-07-06", "1946-08-19",    "1924-06-12",           "1911-02-06", "1924-10-01")))

target_uid_column <- "id"
lookup_uid_column <- "num"

source_nuid_columns <- c("first_name", "middle_name", "last_name", "dob"  )
lookup_nuid_columns <- c("fname",      "mname",       "lname",     "bdate")

In [5]:
target <- source

# Reverses the order of the nuid columns to prioritize the first elements in the final lookup
source_nuid_columns <- rev(source_nuid_columns)
lookup_nuid_columns <- rev(lookup_nuid_columns)

In [6]:
for (target_row in 1:nrow(target)) {
    for (nuid_column_1 in 1:length(source_nuid_columns)) {
        for (nuid_column_2 in nuid_column_1:length(source_nuid_columns)) {
            if (nuid_column_1 != nuid_column_2) {
                if(nrow(filter(lookup_table, 
                            !!sym(lookup_nuid_columns[nuid_column_1]) == target[[target_row,{{source_nuid_columns[nuid_column_1]}}]], 
                            !!sym(lookup_nuid_columns[nuid_column_2]) == target[[target_row,{{source_nuid_columns[nuid_column_2]}}]]))==1) {
                    target[target_row, {{target_uid_column}}] <- filter(lookup_table, 
                            !!sym(lookup_nuid_columns[nuid_column_1]) == target[[target_row,{{source_nuid_columns[nuid_column_1]}}]], 
                            !!sym(lookup_nuid_columns[nuid_column_2]) == target[[target_row,{{source_nuid_columns[nuid_column_2]}}]])[[{{lookup_uid_column}}]]
                }
            }
        }
    }
}

ERROR: Error: `arg` must be a symbol


In [ ]:
target

In [ ]:
sessionInfo()